In [1]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
%run utils.ipynb
%run transform.ipynb
%run model.ipynb



c:\Users\haZAR\anaconda3\envs\pytorchgpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


UNET(
  (ups): ModuleList(
    (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
    (1): DoubleConv(
      (relusig): ReluSIG(
        (gelu): GELU(approximate='none')
        (sigmoid): Sigmoid()
      )
      (conv): Sequential(
        (0): Conv2d(1024, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReluSIG(
          (gelu): GELU(approximate='none')
          (sigmoid): Sigmoid()
        )
        (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReluSIG(
          (gelu): GELU(approximate='none')
          (sigmoid): Sigmoid()
        )
      )
    )
    (2): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    (3): DoubleConv(
      (relusig): ReluSIG(
        (gelu): GELU(approximate=

In [2]:
import torchvision.transforms as transforms

In [5]:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
NUM_EPOCHS = 50
NUM_WORKERS = 0
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 160
PIN_MEMORY = True
LOAD_MODEL = 1
TRAIN_IMG_DIR= "images"
VAL_IMG_DIR = "Validation"
CSV_PATH = "role_challenge_dataset_ground_truth.csv"
VAL_CSV_PATH = "role_challenge_dataset_ground_truth - Copy.csv"
hyp1 = "C:/Users/haZAR/Desktop/Subahnshu_Sethi_ResearchTeam/Landmark Detection/Model_weights/Hypothies1withconcatskip.pth.tar"
hyp2 = "C:/Users/haZAR/Desktop/Subahnshu_Sethi_ResearchTeam/Landmark Detection\Model_weights/Hypothises2onlyGELUSIG.pth.tar"





<>:15: SyntaxWarning: invalid escape sequence '\M'
<>:15: SyntaxWarning: invalid escape sequence '\M'
C:\Users\haZAR\AppData\Local\Temp\ipykernel_6132\896916408.py:15: SyntaxWarning: invalid escape sequence '\M'
  hyp2 = "C:/Users/haZAR/Desktop/Subahnshu_Sethi_ResearchTeam/Landmark Detection\Model_weights/Hypothises2onlyGELUSIG.pth.tar"


In [6]:
def train_fn(loader, model, optimizer, loss_fn):

    loop = tqdm.tqdm(loader)
    losses = []

    for (image,heatmap) in (loop):
        image, heatmap = image.float().to(device = DEVICE), heatmap.float().to(device = DEVICE)
        output = model(image)
        # print(output[1])
        loss = loss_fn(output, heatmap)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        losses.append(loss.item())
        loop.set_postfix(loss=loss.item())

    return sum(losses) / len(losses)
        



In [7]:
def main():
    train_transform = transforms.Compose([Rescale(output_size=(IMAGE_HEIGHT, IMAGE_WIDTH)),
                                          RandomHorizontalFlip(p=0.04),
                                          ToTensor()])
    valid_transform = transforms.Compose([Rescale(output_size=(IMAGE_HEIGHT, IMAGE_WIDTH)),
                                          ToTensor()])

    model = load_model('UNet').to(device= DEVICE)
    loss_fn = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)
 
 

    train_loader , val_loader = get_loaders(
        TRAIN_IMG_DIR,
        CSV_PATH,
        VAL_IMG_DIR,
        VAL_CSV_PATH,
        BATCH_SIZE,
        train_transform,
        valid_transform,
        NUM_WORKERS,
        PIN_MEMORY
    )
    if LOAD_MODEL == 1:
        load_checkpoint(torch.load(hyp1), model)
    elif LOAD_MODEL == 2 :
        load_checkpoint(torch.load(hyp2), model)
    check_accuracy(val_loader,model,loss_fn,device =DEVICE)
    losses = []

    for epochs in range(NUM_EPOCHS):
        model.train()

        loss = train_fn(train_loader, model, optimizer, loss_fn)
        losses.append(loss)

        checkpoint = {
            "state_dict":model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }

        save_checkpoint(checkpoint)

        check_accuracy(val_loader,model,loss_fn,device = DEVICE)

        save_predictions_as_imgs(
            val_loader,model,folder = "saved_images/" ,device = DEVICE

        )
    plt.plot(losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()
        
if __name__ == "__main__":
    main()



    

Validation loss: 0.495901


  3%|▎         | 4/133 [00:06<03:42,  1.73s/it, loss=0.486]


KeyboardInterrupt: 